#Bayesian SN Cosmology

[Rubin et al. 2015](http://arxiv.org/abs/1507.01602) lays out a pretty complete Bayesian model for going from light curve parameters for a large, heterogeneous set of SNe to inferred cosmological parameters. However, it has a couple deficiencies: (1) There's no publicly available code, meaning that the model cannot be modified and improved. (2) The implementation runs slowly (timescale of days). In this hack, we'll start on an open-source package implementing the model and seeing if we can take advantage of aspects of the problem to make it run a whole lot faster.

##Simplistic Model

<img src="Simplified SN Cosmology PGM.jpg">

##Assigning Probability Distributions

This is mostly just following the Rubin et al. 2015 explanation, quoted below...

In [10]:
import scipy as sp
import numpy as np
import sncosmo
#import matplotlib as plt
from astropy.cosmology import FlatLambdaCDM

#%matplotlib inline

"Each simulated dataset has 250 SNe, except the highest-redshift, which has 50."

In [11]:
size_we_want = 250

"We generate the $x_{1}$ population from a unit normal distribution, centered on zero."

In [12]:
x1_true_dist = np.random.normal(loc=0, scale=1, size=size_we_want)

"We assume that the uncertainties on $m_B$, $x_{1}$, and $c$ are 0.05, 0.5, and 0.05, and are uncorrelated."

In [13]:
x1_obs_unc = 0.5*np.ones_like(x1_true_dist)

x1_obs_dist = x1_true_dist + x1_obs_unc*np.random.normal(loc=0, scale=1, size=size_we_want)

"We draw the population $c$ values from the sum of a Gaussian distribution of width 0.1 magnitudes and an exponential with rate 1/(0.1 magnitudes). We center the distribution on zero." $\beta$ is the scale parameter, which is the inverse of the rate parameter $\lambda = 1/\beta$.  The rate parameter is an alternative, widely used parameterization of the exponential distribution.

In [14]:
c_true_norm = np.random.normal(loc=0, scale=0.1, size=size_we_want)
c_true_exp = np.random.exponential(scale=0.1/1, size=size_we_want)
c_true_dist = np.add(c_true_norm, c_true_exp)

c_obs_unc = 0.5*np.ones_like(c_true_dist)

c_obs_dist = c_true_dist + c_obs_unc*np.random.normal(loc=0, scale=1, size=size_we_want)

"We generate four simulated datasets spanning the redshift ranges 0.02-0.05, 0.05-0.4, 0.2-1.0, and 0.7-1.4." We're just doing one set for now, from 0.2-1.0.

In [15]:
z_dist = np.random.uniform(0.2, 1.0, size=size_we_want)

$\alpha$ and $\beta$ are assumed to be constant, with values 0.13 and 3.0, respectively. $M_{B}$ is set to -19.1 and $\Omega_{m}$ is set to 0.3 (flat $\Lambda \text{CDM}$ model).

In [16]:
alpha = 0.13
beta = 3.0
MB = -19.1
Omega_m = 0.3
sigma_int = 0.15

We calculated the $\mu$ values using astropy.cosmology.lambdacdm().

In [17]:
cosmo = FlatLambdaCDM(H0=70, Om0=Omega_m)
mu_dist = cosmo.distmod(z_dist).value

"We assume that the intrinsic dispersion covariance matrix is correct in SALT2, and that only dispersion in $m_{B}$ (gray dispersion) remains. The statistical model does not have access to this information, and fits for the full unknown matrix, overestimating the uncertainties on $x_{1}$ and $c$, and thus slightly biasing $\alpha$ and $\beta$ away from zero (see Section 2.5). (This is not a unique problem for our framework; the old technique would have the same bias.)" We might come back to this but for now we just solved for $m_{B} = M_{B} - {\alpha}x_{1} + {\beta}{c} + \mu$ and introduced some scatter.

In [18]:
mb_true_dist = MB - alpha*x1_true_dist + beta*c_true_dist + mu_dist + np.random.normal(loc=0, scale=sigma_int, size=size_we_want)
 
mb_obs_unc = 0.05*np.ones_like(mb_true_dist)

mb_obs_dist = mb_true_dist + mb_obs_unc*np.random.normal(loc=0, scale=1, size=size_we_want)